In [12]:
import tensorflow as tf
import tflearn
from tensorflow.python.framework import ops
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import os
import re

In [13]:
# training without normalizing the image (scaling)
dataset_dir = 'Dataset'

checkpoint_path = 'TrainedModel/checkpoints/' + 'Gesture12RecognitionModel.tflearn'
best_checkpoint_path = 'TrainedModel/checkpoints/' + 'Gesture12RecognitionModelBest.tflearn'
saved_model_path = 'TrainedModel/' + 'Gesture12RecognitionModel.tflearn'

test_ratio = 0.2
validation_ratio = 0.2
batch_size = 50
epochs = 10

all_images = []
class_ids = []
n_classes = len(os.listdir(dataset_dir))
print(n_classes)

alldirs = []
for d in os.listdir(dataset_dir):
    alldirs.append(f"{dataset_dir}/{d}")
print(alldirs)

12
['Dataset/Direction_left', 'Dataset/Direction_right', 'Dataset/Fist', 'Dataset/Five-palm', 'Dataset/Four', 'Dataset/OK', 'Dataset/One', 'Dataset/Stop', 'Dataset/Three', 'Dataset/Thumbs_down', 'Dataset/Thumbs_up', 'Dataset/Two']


In [14]:
for d in alldirs:
    for img in os.listdir(d):
        label = re.findall('[l]=[0-9]+', img)
        label = label[0][2:]
        
        current_img = f"{d}/{img}"
        image = cv2.imread(current_img)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        all_images.append(gray_image.reshape(89, 100, 1))
        class_ids.append(label)
    print(f"{d} done.")
            
all_images = np.array(all_images)
class_ids = np.array(class_ids)

# print(len(all_images))
# print(len(class_ids))

KeyboardInterrupt: 

In [17]:
x_train, x_test, y_train, y_test = train_test_split(all_images, class_ids, test_size=test_ratio)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_ratio)

In [ ]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
y_validation = to_categorical(y_validation, n_classes)

In [ ]:
ops.reset_default_graph()
#                           [batch, height, width, in_channels]
convnet = input_data(shape=[None, 89, 100, 1], name='input')

convnet = conv_2d(convnet, nb_filter=32, filter_size=2, activation='relu')
convnet = max_pool_2d(convnet,  kernel_size=2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 256, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 256, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, n_units=1000, activation='relu')
convnet = dropout(convnet, keep_prob=0.75)

convnet = fully_connected(convnet, n_units=n_classes, activation='softmax')

convnet = regression(convnet,
                     optimizer='adam',
                     learning_rate=0.001,
                     loss='categorical_crossentropy',
                     name='regression')

model = tflearn.DNN(convnet, 
                    checkpoint_path=checkpoint_path, 
                    best_checkpoint_path=best_checkpoint_path)

In [ ]:
x_train, y_train = shuffle(x_train, y_train, random_state=0)

model.fit(x_train, y_train, n_epoch=epochs,
          batch_size=batch_size, validation_set=(x_validation, y_validation),
        snapshot_step=100, show_metric=True, run_id='convnet_coursera')


print(model.get_summary())

In [ ]:
score = model.evaluate(x_test, y_test)
print(score)

model.save(saved_model_path)